# 2.1

The results of `spark.out` were obtained running the program `spark.c` on lab5p7

In [25]:
import tabulate

NR_ITERATIONS = 100  # defined in spark.c
CACHE_MIN     = 4    # in KiB

sample = [["Cache Size (KiB)", "Avg Elapsed Time (s)", "# Accesses", 
               "Avg Time per Access (ns)"]]

fp = open("spark.out", "r")
next(fp)             # ignore first line
line = fp.readline() # second line, [LOG] line which we ignore, we just 
                     # read it here to initialize the loop condition

cache_size = CACHE_MIN
cum_elapsed_time = 0.0
elapsed_time_obs_count = 0

while line:
    line = fp.readline()
    
    if line.startswith("[LOG]") or line == "":
        avg_elapsed_time = float(cum_elapsed_time) / elapsed_time_obs_count
        # reverse engineering the nr_accesses value: 
        # in spark.c every stride value simulates the same nr of accesses, 
        # considering stride = 1 we can see this is the nr of accesses for 
        # each cache_size
        nr_accesses = cache_size * 2**10 * NR_ITERATIONS

        sample.append([f"{cache_size} KiB", avg_elapsed_time, nr_accesses, 
                           (avg_elapsed_time / nr_accesses) * 10**9])

        cache_size *= 2
        cum_elapsed_time = 0.0
        elapsed_time_obs_count = 0
        continue

    _, _, elapsed_time, *_ = line.split("\t")
    cum_elapsed_time += float(elapsed_time)
    elapsed_time_obs_count += 1


tabulate.tabulate(sample, headers="firstrow", tablefmt="html")

Cache Size (KiB),Avg Elapsed Time (s),# Accesses,Avg Time per Access (ns)
4 KiB,0.00092925,409600,2.26868
8 KiB,0.00188254,819200,2.29802
16 KiB,0.00382021,1638400,2.33167
32 KiB,0.00756427,3276800,2.30843
64 KiB,0.0167127,6553600,2.55015
128 KiB,0.0396387,13107200,3.02419
256 KiB,0.0934102,26214400,3.56332
512 KiB,0.203428,52428800,3.88008
1024 KiB,0.442602,104857600,4.22098
2048 KiB,0.921256,209715200,4.39289


We conclude that the cache capacity is **32 KiB** because that's where we see the first singificant increase in average time per access, prior to array sizes of 64 KiB the time remained somewhat constant. We could be tempted to guess 64 KiB is the cache capacity since 2.55ns to 3.02ns is a much more significant increase, however, looking at the following data in `spark.out`
```
size	stride	elapsed(s)	cycles
(...)
[LOG]: running with array of size 64 KiB
65536	1	    0.014075	14076	
65536	2	    0.013435	13435	
65536	4	    0.015363	15363	
65536	8	    0.015878	15878	
65536	16	    0.015519	15519	
65536	32	    0.015379	15379	
65536	64	    0.013368	13368	
65536	128	    0.013804	13804	
65536	256	    0.013328	13329	
65536	512	    0.013572	13571	
65536	1024    0.013174	13174	
65536	2048    0.028942	28941	
65536	4096    0.035842	35842	
65536	8192    0.015605	15605	
65536	16384   0.015422	15422	
65536	32768   0.014697	14698
```
we can see the elapsed time for stride values of 2048 and 4096 are much bigger than other observations, this indicates the miss rate has inscreased (even if there were still cache misses happening with different stride values), meaning our L1 cache filled up and cannot hold 64 KiB of data.

<table>
  <tr>
    <th>Array Size:</th>
    <td>8 KiB</td>
    <td>16 KiB</td>
    <td>32 KiB</td>
    <td>64 KiB</td>
    <td>128 KiB</td>
  </tr>
  <tr>
    <th>t2 - t1 (s):</th>
    <td>0.00188254</td>
    <td>0.00382021</td>
    <td>0.00756427</td>
    <td>0.0167127 </td>
    <td>0.0396387 </td>
  </tr>
  <tr>
    <th># accesses[i]:</th>
    <td>819200</td>
    <td>1638400</td>
    <td>3276800</td>
    <td>6553600</td>
    <td>13107200</td>
  <tr>
    <th># mean accesse time (ns)</th>
    <td>2.29802</td>
    <td>2.33167</td>
    <td>2.30843</td>
    <td>2.55015</td>
    <td>3.02419</td>
  </tr>
</table>

# 2.2

In the chart, we can identify a group of array sizes whose access time is small, and another group whose access time is significantly higher. We then conclude the cache size to be **64 KiB** because up until that point, all the array values can fit into cache, making the read and write operation times small and relatively constant, regardless of stride. At 128 KiB, all the array content cannot fit simultaneously into cache, resulting in cache misses and, consequently, higher read and write times.


# 2.3

For the arrays whose size exceeds the cache capacity (>= 128 KiB) we can determine the cache block's size by seeing which stride value stabilizes the read and write times. For stride values below 16, the access times increase for each stride step, after 16 the read and write times stabilize because we start accessing contents that fall on different cache blocks and, since the entire array cannot entirely fit into cache, this will result in a 100% miss rate. All subsequent strides values will keep the 100% miss rate.

We can then determine the cache block's size to be **16 Bytes** (since our array is of type `uint8_t`).

# 2.4

For the 64 KiB size array with a stride of 16, which results in a cache hit (as seen in 2.2) we have an access time of ~370ns. For the 256 KiB array with a stride of 16, which results in a cache miss (as seen in 2.3), we have an access time of ~750ns. Then we can conclude that the miss penalty (the time it takes to load the block from a lower level of the memory hierarchy into the cache) is **750ns - 370ns = 380ns**

---

# 3.1.1

### **a)** 
The code section shown above is found in `cm1.c`, more specifically starting at line 41, and serves the purpose 
of testing L1 data cache accesses.
In the `cm1.c` program we are analyzing **PAPI_L1_DCM** processor events which are **L1 Cache Misses**, that is, we are testing how many times we tried to fetch data that was not present in the L1 cache and needed to be loaded into cache from a lower level of the memory hierarchy.

### **b)**
Resulting graph from running `./cm1_proc.sh` with the data found in `cm1.out`
![Average nr of misses and stride variation graph](graph.png)
The *x* axis represents the stride values

### **c)** 
- The L1 cache is **32 KiB**. We can see a significant increase in the miss rate at 64 KiB of array size while the miss rate remains rather constant for lower array size values. In particular, for 32 KiB we have an average miss rate of ~0 (not absolute 0 because of other program values that might be getting stored in cache) meaning this is the highest size of the array that totally fit into cache.

- For the array whose size exceeds the cache capacity (64 KiB) we see the miss rate becoming 1 at the stride value of **64**. This indicates that, at this stride value, we start accessing elements of the array that fall on different blocks of the L1 cache, meaning we will always need to load a new block into the cache for every access, resulting in a 100% miss rate. Since our array is of type `uint8_t` this means the cache block must be **64 B** in size. For smaller stride values, say of 32, we can see the miss rate decreases, this is because we make two consequent accesses to data in the same block, only needing to load data into the cache half of the times.

-

---

# 3.1.2

### **a)** 
In order to analyze the characteristics of the L2 Cache we changed the events type count from **PAPI_L1_DCM** to **PAPI_L2_DCM**. We also changed the **CACHE_MIN** value to 64 KiB since this was the first value with which we started having cache misses on L1 cache and also changed **CACHE_MAX** to 1 MiB.

This way we can analyze the L2 Cache misses using PAPI, the **CACHE_MAX** value was increased because L2 Cache is much bigger and lower values might not trigger cache misses.

### **b)** 

![Average number of misses in L2](cm2.png)


### **c)** 
- We can see two big increases in average misses, one at 256KiB and another one at 512KiB. The jump from 256 to 512 is much more significant, indicating our L2 might have been fully filled. We can then conclude our L2 cache capacity to be **256KiB**
- I have no idea what the block size is
- How do I check associativity rules?!

---

# 3.2.1

### **a)** 
Each matrix has 512 by 512 entries, each entry of size 2B (values of type int16_t), then each matrix will take up $512^2 * 2$B = $2^{19}$ B = **512 KiB**

### **b)**
```
After resetting counter 'PAPI_L1_DCM' [x10^6]: 0.000000
After resetting counter 'PAPI_LD_INS' [x10^6]: 0.000000
After resetting counter 'PAPI_SR_INS' [x10^6]: 0.000000
After stopping counter 'PAPI_L1_DCM'  [x10^6]: 135.052581
After stopping counter 'PAPI_LD_INS'  [x10^6]: 402.653833
After stopping counter 'PAPI_SR_INS'  [x10^6]: 134.218051
Wall clock cycles [x10^6]: 716.400162
Wall clock time [seconds]: 0.210208
Matrix checksum: 2717908992

```

In [47]:
sf_l1_cache_misses = 135.052581
sf_ld_operations = 402.653833
sf_sr_operations = 134.218051
sf_total_mem_instructions = ld_operations + sr_operations
sf_clock_cycles = 716.400162
sf_wall_clock_time = 0.210208

print(f"Total number of L1 data cache misses: {sf_l1_cache_misses} x 10^6")
print(f"Total number of load / store instructions completed: {sf_total_mem_instructions} x 10^6")
print(f"Total number of clock cycles: {sf_clock_cycles} x10^6")
print(f"Elapsed time: {sf_wall_clock_time} seconds")

Total number of L1 data cache misses: 135.052581 x 10^6
Total number of load / store instructions completed: 536.871884 x 10^6
Total number of clock cycles: 716.400162 x10^6
Elapsed time: 0.210208 seconds


### **c)**

In [48]:
sf_miss_rate = sf_l1_cache_misses / sf_total_mem_instructions
sf_hit_rate = 1 - sf_miss_rate

print(f"Hit rate = {round(sf_hit_rate, 3) * 100}%")

Hit rate = 74.8%


---

# 3.2.2

### **a)**
```
After resetting counter 'PAPI_L1_DCM' [x10^6]: 0.000000
After resetting counter 'PAPI_LD_INS' [x10^6]: 0.000000
After resetting counter 'PAPI_SR_INS' [x10^6]: 0.000000
After stopping counter 'PAPI_L1_DCM'  [x10^6]: 4.217954
After stopping counter 'PAPI_LD_INS'  [x10^6]: 402.653833
After stopping counter 'PAPI_SR_INS'  [x10^6]: 134.218051
Wall clock cycles [x10^6]: 656.802860
Wall clock time [seconds]: 0.192722
Matrix checksum: 2717908992
````

In [40]:
l1_cache_misses = 4.217954
ld_operations = 402.653833
sr_operations = 134.218051
total_mem_instructions = ld_operations + sr_operations
clock_cycles = 656.802860
wall_clock_time = 0.192722

print(f"Total number of L1 data cache misses: {l1_cache_misses} x 10^6")
print(f"Total number of load / store instructions completed: {total_mem_instructions} x 10^6")
print(f"Total number of clock cycles: {clock_cycles} x10^6")
print(f"Elapsed time: {wall_clock_time} seconds")

Total number of L1 data cache misses: 4.217954 x 10^6
Total number of load / store instructions completed: 536.871884 x 10^6
Total number of clock cycles: 656.80286 x10^6
Elapsed time: 0.192722 seconds


### **b)**

In [41]:
miss_rate = l1_cache_misses / total_mem_instructions
hit_rate = 1 - miss_rate

print(f"Hit rate = {hit_rate}")

Hit rate = 0.9921434626664115


### **c)**

```
After resetting counter 'PAPI_L1_DCM' [x10^6]: 0.000000
After resetting counter 'PAPI_LD_INS' [x10^6]: 0.000000
After resetting counter 'PAPI_SR_INS' [x10^6]: 0.000000
After stopping counter 'PAPI_L1_DCM'  [x10^6]: 4.486579
After stopping counter 'PAPI_LD_INS'  [x10^6]: 402.915978
After stopping counter 'PAPI_SR_INS'  [x10^6]: 134.480196
Wall clock cycles [x10^6]: 626.630824
Wall clock time [seconds]: 0.183868
Matrix checksum: 2717908992
````

In [42]:
tp_l1_cache_misses = 4.486579
tp_ld_operations = 402.915978
tp_sr_operations = 134.480196
tp_total_mem_instructions = tp_ld_operations + tp_sr_operations
tp_clock_cycles = 626.630824
tp_wall_clock_time = 0.183868

print(f"Total number of L1 data cache misses: {tp_l1_cache_misses} x 10^6")
print(f"Total number of load / store instructions completed: {tp_total_mem_instructions} x 10^6")
print(f"Total number of clock cycles: {tp_clock_cycles} x10^6")
print(f"Elapsed time: {tp_wall_clock_time} seconds")

Total number of L1 data cache misses: 4.486579 x 10^6
Total number of load / store instructions completed: 537.396174 x 10^6
Total number of clock cycles: 626.630824 x10^6
Elapsed time: 0.183868 seconds


In [43]:
tp_miss_rate = tp_l1_cache_misses / tp_total_mem_instructions
tp_hit_rate = 1 - tp_miss_rate

print(f"Hit rate = {tp_hit_rate}")

Hit rate = 0.9916512635983151


The difference isn't much significant because the matrix transposition doesn't weight in as much as matrix multiplication on the runtime

### **d)**

In [51]:
diff = tp_hit_rate - sf_hit_rate
print(f"Difference in Hit Rate: {diff}")

Difference in Hit Rate: 0.24320584305176252


In [52]:
speedup = sf_wall_clock_time / tp_wall_clock_time
print(f"Speedup: {speedup}")

Speedup: 1.1432549437640047


There was a slight speedup using the transposition, although not super significant. This might indicate the miss penalty was low since the 24% improved hit rate only resulted in a 1.14 speedup. This improvement might not be good, in terms of memory usage (since we need 512KiB aditional bytes in memory) compared to time improvement